# <center style="color:white; padding: 10px; border-radius:5px 5px; background-image: url(https://i.imgur.com/0fetaJr.png);">TPS - Jan 2022 - Linear Regression (Few and simple steps with XGBRegressor)</center>

<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F59561%2Fcc6d9a21f0c3ed71b00113b33efb2b66%2Fkaggle_sweater.png?generation=1640900016906235&alt=media" /></center>

# <center style="background-color:#4267B2; color:white; padding: 10px; border-radius:5px 5px;">Basic setup and import</center>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from xgboost import XGBRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# <center style="background-color:#4267B2; color:white; padding: 10px; border-radius:5px 5px;">Load data to dataframe</center>

In [ ]:
df_train_full = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
df_test_full = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

df = df_train_full.copy()
df_test = df_test_full.copy()

<div style="background-color:#2d862d; color:white; padding: 5px 5px 5px 15px; border-radius:5px 5px; font-size:18px;">Remove outlier data</div>

In [ ]:
df = df.drop(df[df['num_sold'] > 915].index)

<div style="background-color:#2d862d; color:white; padding: 5px 5px 5px 15px; border-radius:5px 5px; font-size:18px;">Convert datetime to different columns</div>

In [ ]:
df['date']=pd.to_datetime(df['date'])
df.insert(loc=2, column='year', value=df['date'].dt.year)
df.insert(loc=3, column='month', value=df['date'].dt.month)
df.insert(loc=4, column='day', value=df['date'].dt.day)

df = df.drop(['date', 'row_id'], axis=1)
df = df.reset_index(drop=True)

#################### Below code is for test data ####################
df_test['date']=pd.to_datetime(df_test['date'])
df_test.insert(loc=2, column='year', value=df_test['date'].dt.year)
df_test.insert(loc=3, column='month', value=df_test['date'].dt.month)
df_test.insert(loc=4, column='day', value=df_test['date'].dt.day)

df_test = df_test.drop(['date', 'row_id'], axis=1)
df_test = df_test.reset_index(drop=True)

<div style="background-color:#2d862d; color:white; padding: 5px 5px 5px 15px; border-radius:5px 5px; font-size:18px;">Feature Engineering and Create dummy columns</div>

In [ ]:
df1 = df['num_sold']
df2 = np.log(df1)

df['num_sold'] = np.log(df['num_sold'])

df['year'] = df['year'].astype('category')
df['month'] = df['month'].astype('category')
df['day'] = df['day'].astype('category')

df = pd.get_dummies(df)

#################### Below code is for test data ####################
df_test['year'] = df_test['year'].astype('category')
df_test['month'] = df_test['month'].astype('category')
df_test['day'] = df_test['day'].astype('category')

df_test = pd.get_dummies(df_test)

df_test.insert(loc=0, column='year_2015', value=0)
df_test.insert(loc=1, column='year_2016', value=0)
df_test.insert(loc=2, column='year_2017', value=0)
df_test.insert(loc=3, column='year_2018', value=1)

df_test = df_test.drop(['year_2019'], axis=1)

<div style="background-color:#2d862d; color:white; padding: 5px 5px 5px 15px; border-radius:5px 5px; font-size:18px;">Test / Train Split of Data</div>

In [ ]:
Y = df[['num_sold']]
X = df.drop(['num_sold'], axis=1)

tr_size = len(X)
tr_size = int(tr_size)

X_train = X.values[0 : tr_size]
Y_train = Y.values[0 : tr_size]

<div style="background-color:#2d862d; color:white; padding: 5px 5px 5px 15px; border-radius:5px 5px; font-size:18px;">Create Model and Check Score</div>

In [ ]:
std_reg = XGBRegressor()
std_reg.fit(X_train, Y_train)

r2_train = std_reg.score(X_train, Y_train)
print("Score:")
print(r2_train)

# Create Y Predictions
Y_predict = std_reg.predict(df_test)

Y_predict_e = []

for data in Y_predict:
    Y_predict_e.append(int(math.exp(data) + (math.exp(data) / 100) * 2.5))
    #Y_predict_e.append(int(math.exp(data)))

<div style="background-color:#2d862d; color:white; padding: 5px 5px 5px 15px; border-radius:5px 5px; font-size:18px;">Generate oufput file for submission</div>

In [ ]:
output = pd.DataFrame({'row_id': df_test_full['row_id'],
                       'num_sold': Y_predict_e})
output.to_csv('submission.csv', index=False)

# <center style="color:white; padding: 10px; border-radius:5px 5px; background-image: url(https://i.imgur.com/0fetaJr.png);">Thank You</center>